In [3]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [4]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/vfnet/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco.py')

PREFIX = '../../input/data/'


# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=90
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco'

cfg.model.bbox_head.num_classes = 11
cfg.optimizer = dict(type='Adam', lr=1e-4, weight_decay=0.0001)
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [7]:
print(cfg.data.train.pipeline)

[{'type': 'LoadImageFromFile'}, {'type': 'LoadAnnotations', 'with_bbox': True}, {'type': 'Resize', 'img_scale': (512, 512), 'multiscale_mode': 'range', 'keep_ratio': True}, {'type': 'RandomFlip', 'flip_ratio': 0.5}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'DefaultFormatBundle'}, {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels']}]


In [1]:
print(cfg.lr_config)

NameError: name 'cfg' is not defined

In [7]:
model = build_detector(cfg.model)

2021-05-11 05:12:13,648 - mmdet - INFO - load model from: open-mmlab://resnext101_64x4d
2021-05-11 05:12:13,649 - mmdet - INFO - Use load_from_openmmlab loader
2021-05-11 05:12:14,161 - mmdet - WARNING - The model and loaded state dict do not match exactly

missing keys in source state_dict: layer2.0.conv2.conv_offset.weight, layer2.0.conv2.conv_offset.bias, layer2.1.conv2.conv_offset.weight, layer2.1.conv2.conv_offset.bias, layer2.2.conv2.conv_offset.weight, layer2.2.conv2.conv_offset.bias, layer2.3.conv2.conv_offset.weight, layer2.3.conv2.conv_offset.bias, layer3.0.conv2.conv_offset.weight, layer3.0.conv2.conv_offset.bias, layer3.1.conv2.conv_offset.weight, layer3.1.conv2.conv_offset.bias, layer3.2.conv2.conv_offset.weight, layer3.2.conv2.conv_offset.bias, layer3.3.conv2.conv_offset.weight, layer3.3.conv2.conv_offset.bias, layer3.4.conv2.conv_offset.weight, layer3.4.conv2.conv_offset.bias, layer3.5.conv2.conv_offset.weight, layer3.5.conv2.conv_offset.bias, layer3.6.conv2.conv_offset.

In [8]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=5.16s)
creating index...
index created!


In [17]:
datasets

In [9]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-11 05:12:28,009 - mmdet - INFO - Start running, host: root@bc720df3e041, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco
2021-05-11 05:12:28,010 - mmdet - INFO - workflow: [('train', 1)], max: 24 epochs


Done (t=0.95s)
creating index...
index created!


2021-05-11 05:14:09,151 - mmdet - INFO - Epoch [1][50/655]	lr: 1.882e-05, eta: 8:48:15, time: 2.023, data_time: 0.050, memory: 5644, loss_cls: 0.9522, loss_bbox: 0.7453, loss_bbox_rf: 1.0050, loss: 2.7025, grad_norm: 14.8900
2021-05-11 05:15:48,037 - mmdet - INFO - Epoch [1][100/655]	lr: 2.782e-05, eta: 8:40:43, time: 1.978, data_time: 0.007, memory: 5644, loss_cls: 0.8386, loss_bbox: 0.6872, loss_bbox_rf: 0.9199, loss: 2.4457, grad_norm: 12.4349
2021-05-11 05:17:27,912 - mmdet - INFO - Epoch [1][150/655]	lr: 3.682e-05, eta: 8:38:49, time: 1.998, data_time: 0.007, memory: 5644, loss_cls: 0.9072, loss_bbox: 0.6557, loss_bbox_rf: 0.8751, loss: 2.4380, grad_norm: 13.6231
2021-05-11 05:19:06,164 - mmdet - INFO - Epoch [1][200/655]	lr: 4.582e-05, eta: 8:34:56, time: 1.965, data_time: 0.006, memory: 5644, loss_cls: 0.8867, loss_bbox: 0.6201, loss_bbox_rf: 0.8251, loss: 2.3319, grad_norm: 12.6583
2021-05-11 05:20:45,022 - mmdet - INFO - Epoch [1][250/655]	lr: 5.482e-05, eta: 8:32:34, time: 1.

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 8.4 task/s, elapsed: 78s, ETA:     0s

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 8.2 task/s, elapsed: 80s, ETA:     0s

2021-05-11 14:33:51,711 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.66s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=6.86s).
Accumulating evaluation results...


2021-05-11 14:34:00,672 - mmdet - INFO - Epoch(val) [24][655]	bbox_mAP: 0.2130, bbox_mAP_50: 0.3040, bbox_mAP_75: 0.2190, bbox_mAP_s: 0.0330, bbox_mAP_m: 0.1840, bbox_mAP_l: 0.4150, bbox_mAP_copypaste: 0.213 0.304 0.219 0.033 0.184 0.415


DONE (t=1.33s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.213
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.304
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.219
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.033
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.184
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.415
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.396
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.396
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.396
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.123
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.377
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.666


In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [5]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/vfnet/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco.py')

PREFIX = '../../input/data/'

epoch = 24

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=90
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco'

cfg.model.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'lastest.pth')

In [6]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [7]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2021-05-13 06:12:40,036 - mmdet - INFO - load model from: open-mmlab://resnext101_64x4d
2021-05-13 06:12:40,037 - mmdet - INFO - Use load_from_openmmlab loader
2021-05-13 06:12:40,289 - mmdet - WARNING - The model and loaded state dict do not match exactly

missing keys in source state_dict: layer2.0.conv2.conv_offset.weight, layer2.0.conv2.conv_offset.bias, layer2.1.conv2.conv_offset.weight, layer2.1.conv2.conv_offset.bias, layer2.2.conv2.conv_offset.weight, layer2.2.conv2.conv_offset.bias, layer2.3.conv2.conv_offset.weight, layer2.3.conv2.conv_offset.bias, layer3.0.conv2.conv_offset.weight, layer3.0.conv2.conv_offset.bias, layer3.1.conv2.conv_offset.weight, layer3.1.conv2.conv_offset.bias, layer3.2.conv2.conv_offset.weight, layer3.2.conv2.conv_offset.bias, layer3.3.conv2.conv_offset.weight, layer3.3.conv2.conv_offset.bias, layer3.4.conv2.conv_offset.weight, layer3.4.conv2.conv_offset.bias, layer3.5.conv2.conv_offset.weight, layer3.5.conv2.conv_offset.bias, layer3.6.conv2.conv_offset.

Use load_from_local loader


OSError: ./work_dirs/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco/lastest.pth is not a checkpoint file

In [14]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 8.0 task/s, elapsed: 105s, ETA:     0s

In [16]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.27585107 169.44269 76.139336 255.18983 167...,batch_01_vt/0021.jpg
1,0 0.127446 273.75284 247.20534 308.4707 265.41...,batch_01_vt/0028.jpg
2,0 0.091894604 70.023026 170.12476 374.02173 37...,batch_01_vt/0031.jpg
3,1 0.060998674 297.3724 158.02838 352.31287 222...,batch_01_vt/0032.jpg
4,0 0.35852015 201.36984 277.01688 260.86188 305...,batch_01_vt/0070.jpg
